# Scrapping and merging code
- This notebook gives the code used to scrap the target (the success rate at the 'bervet des collèges' from the site 'Le journal des Femmes'
- it also gives the merge procedure with a table with a lot of data per college from data_gouv. 
- Finally there is a small cleaning code for some macro socio economic data per cities
- More information about the data is provided in the starting_kit_notebook

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_rows", 500)
import re
import recordlinkage as rl

## Geographic data

- a table with some socio economic data per city

In [ ]:
# donnees per city 

cities_data = pd.read_excel("./data//donnees_geographiques/insee_communes/base_cc_comparateur.xls", sheet_name = 0, header = 4, skiprows = 1)
arrondissements_data = pd.read_excel("./data/donnees_geographiques/insee_communes/base_cc_comparateur.xls", sheet_name = 1, header = 4, skiprows = 1)
#drop Paris, Lyon and Marseilles in cities_data
cities_data = cities_data.drop(cities_data[cities_data.LIBGEO.isin(["Paris", "Lyon", "Marseille"])].index)
cities_data = pd.concat([cities_data, arrondissements_data], ignore_index=True)


In [ ]:
cities_data.head()

In [ ]:

#correspondance between the postal code and the insee code
cp_ci = pd.read_csv("./data/donnees_geographiques/donnees_postal_code/code-postal-code-insee-2015.csv", sep = ";")

In [ ]:
cities_data.CODGEO = cities_data.CODGEO.astype(str) #codgeo = insee code
mapper_insee_cp = dict(zip(cp_ci.INSEE_COM, cp_ci.Code_postal.apply(lambda x: "nan" if x !=x else str(x)[:-2])))
cities_data["postal_code"] = cities_data.CODGEO.apply(lambda x: mapper_insee_cp[x] if x in mapper_insee_cp.keys() else "nan" )

cities_data.rename(columns = {"CODGEO":"insee_code", "P15_POP":"population","MED15":"med_std_living", "TP6015":"poverty_rate",
                              "P15_CHOM1564":"nb_unemployed", "P15_ACT1564":"labour_force"}, inplace=True)
cities_data['unemployment_rate'] = cities_data['nb_unemployed']/cities_data['labour_force']

In [ ]:
cities_data.head()

In [ ]:
cities_data.to_csv("./data/donnees_geographiques/cities_data.csv")

### Scrapping de la target : Taux moyen de reussite au brevet sur 3 ans 

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

### Scrapping journal des femmes
- we scrapped the name of the college, its city and success rate
- WARNING : the code is quite long to run (one hour)
 

In [ ]:
# code scrapping 
url = 'https://www.journaldesfemmes.fr/maman/ecole/classement/colleges/taux-reussite-brevet?'
html = urlopen(url)
soup = BeautifulSoup(html, "html")
last_page = 120

colleges = []
target = [] # the success rate
villes = []

for p in range(1, last_page +1):
    url2 = url+'page='+str(p)
    html2 = urlopen(url2)
    soup2 = BeautifulSoup(html2, "html")
    res = [i for i in soup2.find_all(['tr'])][1:] # not interested by the first row
    for i in res:
        #get the name
        colleges.append(i.find_all('td')[1].text)
        #get the target 
        target.append(i.find_all('td')[2].text)
        
        #get the city
        url_college = 'https://www.journaldesfemmes.fr'+ i.find('a').attrs['href']
        html_college = urlopen(url_college)
        soup_college = BeautifulSoup(html_college, 'html')
        villes.append(soup_college.find_all('tr')[3].find('a').text)
        assert len(colleges) == len(target)
        assert len(colleges) == len(villes)
        
        
       

In [ ]:
data_colleges = pd.DataFrame({'Name':colleges,
                             'target':target,
                             'ville':villes})

In [ ]:
def treat_rate(x):
    x = re.sub('\s','', x)
    x = re.sub('%','', x)
    x = re.sub(',','.', x)
    return np.float(x)
 
data_colleges['target'] = data_colleges['target'].apply(treat_rate)

In [ ]:
def treat_college(x):
    x = re.sub('Collège ', '', x)
    return x


In [ ]:
data_colleges.head()

In [ ]:
#to csv 
data_colleges.to_csv('./data/college/target.csv')

## Merging

- We merge the target data frame with a table found on (source <a href= 'https://www.data.gouv.fr/fr/datasets/colleges-education-prioritaire/'> data.gouv </a> ) with a lot of informations about the college
- As a lot of colleges have the same name, we also need the city to do the merge.
- We use the clean function of the record linkage 

In [ ]:
# target data
target = pd.read_csv('./data/college/target.csv', index_col=0)
target.head()

In [ ]:
# data of data.gouv
rep_data = pd.read_csv("./data/college/fr-en-colleges-ep.csv", sep=';')
# we only select the 2017 year as this is the year of the ranking on the journal des femmes website
rep_data = rep_data[rep_data['Rentrée scolaire'] == 2017]
print('shape', rep_data.shape)
rep_data.rename(columns={'Patronyme':'Name', 'Commune nom':'City_name'}, inplace=True)
rep_data.head()


In [ ]:
from recordlinkage.preprocessing import clean

In [ ]:
target.Name = clean(target.Name, strip_accents='unicode') # remove accent
target.Name = target.Name.apply(lambda x: re.sub('college ', '', x)) #remove the word college
target.Name = target.Name.apply(lambda x:re.sub('prive ', '', x)) #remove the word prive
target.ville = clean(target.ville, strip_accents='unicode')
rep_data.Name = clean(rep_data.Name)
rep_data.City_name = clean(rep_data.City_name)

In [ ]:
target.head()

In [ ]:
data_college = pd.merge(rep_data, target, left_on=['Name', 'City_name'], right_on=['Name', 'ville'], how='inner')
data_college.drop(columns=['ville'], axis=1, inplace=True) #drop the ville and keep city_name

In [ ]:
data_college.head()

In [ ]:
data_college.shape

In [ ]:
drop_col = ["Identifiant de l'établissement dans la base",
            'Rentrée scolaire',
           'Numéro établissement',
            'Type établissement code',
           'Type établissement nom',
            'Secteur code', #all public
            'Secteur nom'
           ]

data_college.drop(columns=drop_col, axis=1, inplace=True)

In [ ]:
data_college.to_csv('./data/college/data_college.csv') #merge df 